In [2]:
# pull in quality data from Wikimedia
from paws.EpochFail import wikiquality as wq

In [3]:
# check to make sure we got what we expected
print(next(wq.read_rows('en')))

{'title': 'Lost Moon', 'prediction': 'Stub', 'weighted_sum': 0.10413085831157592, 'page_id': 3204489, 'timestamp': '20051201000000', 'rev_id': 28847753}


In [4]:
# here's what those 'predicted' values mean: 
# https://meta.wikimedia.org/wiki/Objective_Revision_Evaluation_Service#Article_quality_models


In [5]:
# look for the women scientists
print(next(wq.read_page_ids('women scientists')))

{'page_id': 9622178}


In [6]:
women_scientists = set()

In [7]:
generator = wq.read_page_ids('women scientists')

for page in generator:
        women_scientists.add(page['page_id'])

In [9]:
with open('women_scientist_pageids.txt', 'w') as f:
    for p in women_scientists:
        f.write(str(p) + '\n')

In [10]:
print(next(wq.read_aq('en')))

{'title': 'Lost Moon', 'prediction': 'Stub', 'weighted_sum': 0.10413085831157592, 'page_id': 3204489, 'timestamp': '20051201000000', 'rev_id': 28847753}


In [11]:
monthly_wiki_quality = {}

women_scientist_aq = (row for row in wq.read_aq('en') if row['page_id'] in women_scientists)

In [12]:
women_scientist_aq

<generator object <genexpr> at 0x7fddac2cf900>

In [ ]:
for i, row in enumerate(women_scientist_aq):
    if row['timestamp'] not in monthly_wiki_quality:
        monthly_wiki_quality[row['timestamp']] = {
            'weighted_sum': 0.0, 
            'weighted_log_sum': 0.0, 
            'n': 0, 
            'predictions': defaultdict(int)
        }
    monthly_wiki_quality[row['timestamp']]['n'] += 1
    monthly_wiki_quality[row['timestamp']]['weighted_sum'] += row['weighted_sum']
    monthly_wiki_quality[row['timestamp']]['weighted_log_sum'] += log(row['weighted_sum'])
    monthly_wiki_quality[row['timestamp']]['predictions'][row['prediction']] += 1
    if i % 10 == 0:
        sys.stderr.write(".")
    if i != 0 and i % 10**3 == 0:
        sys.stderr.write("\n")